In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /anaconda3/lib/python3.7/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [33]:
# Set features. This will also be used as your x values.

selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
        'koi_time0bk', 'koi_time0bk_err1','koi_impact', 'koi_impact_err1','koi_duration', 'koi_duration_err1',
        'koi_depth', 'koi_depth_err1','koi_prad', 'koi_prad_err1', 'koi_teq', 'koi_insol', 'koi_insol_err1',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1','koi_slogg', 'koi_slogg_err1', 
       'koi_srad', 'koi_srad_err1',  'ra', 'dec','koi_kepmag']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [34]:
target = df['koi_disposition'].values.reshape(-1, 1)
target

array([['CONFIRMED'],
       ['FALSE POSITIVE'],
       ['FALSE POSITIVE'],
       ...,
       ['CANDIDATE'],
       ['FALSE POSITIVE'],
       ['FALSE POSITIVE']], dtype=object)

In [35]:
# split train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)

In [36]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_time0bk,koi_time0bk_err1,koi_impact,koi_impact_err1,...,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_slogg,koi_slogg_err1,koi_srad,koi_srad_err1,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,7.380000e-05,133.077240,0.008440,0.150,0.305,...,1,5737,154,4.327,0.153,1.125,0.310,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,6.060000e-06,132.020050,0.007950,0.291,0.193,...,1,5855,158,4.578,0.033,0.797,0.211,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,6.540000e-05,134.460380,0.006190,0.970,0.879,...,1,6328,151,4.481,0.050,0.963,0.290,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,1.910000e-05,174.662240,0.001820,0.300,0.145,...,1,4768,76,4.536,0.056,0.779,0.023,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,5.150000e-07,172.258529,0.000083,0.831,0.016,...,1,5712,77,4.359,0.110,1.082,0.173,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [37]:
# Scale your data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [38]:
# encode target data

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Train the Model



In [39]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train_scaled, encoded_y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [41]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9056064073226545


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [42]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [100, 200, 500, 1000],
              'min_samples_split': [2, 3, 4], 
             "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

grid = GridSearchCV(model, param_grid, verbose=3)

In [43]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] bootstrap=True, criterion=gini, min_samples_split=2, n_estimators=100 
[CV]  bootstrap=True, criterion=gini, min_samples_split=2, n_estimators=100, score=0.906232132647227, total=   0.9s
[CV] bootstrap=True, criterion=gini, min_samples_split=2, n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  bootstrap=True, criterion=gini, min_samples_split=2, n_estimators=100, score=0.8924485125858124, total=   0.8s
[CV] bootstrap=True, criterion=gini, min_samples_split=2, n_estimators=100 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s remaining:    0.0s


...

[CV]  bootstrap=True, criterion=entropy, min_samples_split=2, n_estimators=1000, score=0.9068038879359634, total=  10.5s
[CV] bootstrap=True, criterion=entropy, min_samples_split=2, n_estimators=1000 
[CV]  bootstrap=True, criterion=entropy, min_samples_split=2, n_estimators=1000, score=0.8924485125858124, total=  10.5s
[CV] bootstrap=True, criterion=entropy, min_samples_split=2, n_estimators=1000 
[CV]  bootstrap=True, criterion=entropy, min_samples_split=2, n_estimators=1000, score=0.8831615120274914, total=  10.3s
[CV] bootstrap=True, criterion=entropy, min_samples_split=3, n_estimators=100 
[CV]  bootstrap=True, criterion=entropy, min_samples_split=3, n_estimators=100, score=0.9068038879359634, total=   1.1s
[CV] bootstrap=True, criterion=entropy, min_samples_split=3, n_estimators=100 
[CV]  bootstrap=True, criterion=entropy, min_samples_split=3, n_estimators=100, score=0.8918764302059496, total=   1.0s
[CV] bootstrap=True, criterion=entropy, min_samples_split=3, n_estimators=100 


[CV]  bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=200, score=0.9056603773584906, total=   2.5s
[CV] bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=200 
[CV]  bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=200, score=0.8867276887871853, total=   2.5s
[CV] bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=200 
[CV]  bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=200, score=0.8837342497136311, total=   2.4s
[CV] bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=500 
[CV]  bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=500, score=0.9050886220697542, total=   6.1s
[CV] bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=500 
[CV]  bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=500, score=0.8907322654462243, total=   6.5s
[CV] bootstrap=False, criterion=gini, min_samples_split=3, n_estimators=500 
[CV]  bootstrap=False, cr

[CV]  bootstrap=False, criterion=entropy, min_samples_split=3, n_estimators=1000, score=0.9079473985134362, total=  16.0s
[CV] bootstrap=False, criterion=entropy, min_samples_split=3, n_estimators=1000 
[CV]  bootstrap=False, criterion=entropy, min_samples_split=3, n_estimators=1000, score=0.8907322654462243, total=  16.0s
[CV] bootstrap=False, criterion=entropy, min_samples_split=3, n_estimators=1000 
[CV]  bootstrap=False, criterion=entropy, min_samples_split=3, n_estimators=1000, score=0.8837342497136311, total=  15.9s
[CV] bootstrap=False, criterion=entropy, min_samples_split=4, n_estimators=100 
[CV]  bootstrap=False, criterion=entropy, min_samples_split=4, n_estimators=100, score=0.9050886220697542, total=   1.6s
[CV] bootstrap=False, criterion=entropy, min_samples_split=4, n_estimators=100 
[CV]  bootstrap=False, criterion=entropy, min_samples_split=4, n_estimators=100, score=0.8941647597254004, total=   1.6s
[CV] bootstrap=False, criterion=entropy, min_samples_split=4, n_estima

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 13.3min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100, 200, 500, 1000], 'min_samples_split': [2, 3, 4], 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [44]:
print(grid.best_params_)
print(grid.best_score_)

{'bootstrap': False, 'criterion': 'entropy', 'min_samples_split': 2, 'n_estimators': 100}
0.897196261682243


In [46]:
model2 = RandomForestClassifier(bootstrap = False, criterion = 'entropy', min_samples_split= 2, n_estimators= 100)
model2.fit(X_train_scaled, encoded_y_train)

RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

# Save the Model

In [47]:
print(f"Training Data Score: {model2.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9021739130434783


In [48]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'random_forest.sav'
joblib.dump(model2, filename)

['random_forest.sav']